Indexing dataset
================

In [1]:
from pathlib import Path
import sys
debug_local = True#False
local = (Path("..") / "indexpaper").resolve()
if debug_local and local.exists():
  sys.path.insert(0, Path("..").as_posix())
  print("extending pathes with local indexpaper")
  print(sys.path)
  %load_ext autoreload
  %autoreload 2

extending pathes with local indexpaper
['..', '/home/antonkulaga/sources/indexpaper/notebooks', '/home/antonkulaga/sources/indexpaper', '/home/antonkulaga/micromamba/envs/indexpaper/lib/python310.zip', '/home/antonkulaga/micromamba/envs/indexpaper/lib/python3.10', '/home/antonkulaga/micromamba/envs/indexpaper/lib/python3.10/lib-dynload', '', '/home/antonkulaga/.local/lib/python3.10/site-packages', '/home/antonkulaga/micromamba/envs/indexpaper/lib/python3.10/site-packages']


In [2]:
import polars as pl
from indexpaper import index

from pycomfort.files import *
from typing import List
from pathlib import Path
from polars import Config
Config.set_fmt_str_lengths(5000)

/home/antonkulaga/micromamba/envs/indexpaper/lib/python3.10/site-packages/beartype/_util/hint/pep/utilpeptest.py:311: BeartypeDecorHintPep585DeprecationWarning: PEP 484 type hint typing.Callable[[list[langchain.schema.document.Document]], NoneType] deprecated by PEP 585. This hint is scheduled for removal in the first Python version released after October 5th, 2025. To resolve this, import this hint from "beartype.typing" rather than "typing". For further commentary and alternatives, see also:
    https://beartype.readthedocs.io/en/latest/api_roar/#pep-585-deprecations
  warn(


polars.config.Config

In [3]:
from indexpaper.paperset import Paperset
from indexpaper.resolvers import *

chunk_size = 512
embeddings, splitter = resolve_embedding_splitter(EmbeddingType.HuggingFace, EmbeddingModels.biolord.value, chunk_size=chunk_size)
from indexpaper.indexing import init_qdrant
db = init_qdrant("biolord_moskalev_papers", path_or_url="http://localhost:6333", embeddings=embeddings)

No sentence-transformers model found with name /home/antonkulaga/.cache/torch/sentence_transformers/FremyCompany_BioLORD-STAMB2-v1. Creating a new one with MEAN pooling.
2023-08-06 14:25:18.677 | INFO     | indexpaper.indexing:init_qdrant:72 - initializing quadrant database at http://localhost:6333


768


In [ ]:
db.add_documents()

# Paperset #

In [4]:
moskalev = Paperset("longevity-genie/moskalev_papers", splitter=splitter)
moskalev.lazy_frame.collect().head(1)

corpusid,content_source_oainfo_openaccessurl,updated,externalids_doi,externalids_pubmed,annotations_abstract,annotations_author,annotations_title,annotations_paragraph
i64,str,str,str,str,list[str],list[str],list[str],list[str]
259168060,null,"""2023-07-04T07:35:35Z""","""10.3389/fnagi.2023.1179988""","""37396658""","[""Song W (2023) The glymphatic system: a new perspective on brain diseases.The glymphatic system is a brain-wide perivascular pathway driven by aquaporin-4 on the endfeet of astrocytes, which can deliver nutrients and active substances to the brain parenchyma through periarterial cerebrospinal fluid (CSF) influx pathway and remove metabolic wastes through perivenous clearance routes. This paper summarizes the composition, overall fluid flow, solute transport, related diseases, affecting factors, and preclinical research methods of the glymphatic system. In doing so, we aim to provide direction and reference for more relevant researchers in the future.""]","[""Alexey Moskalev "", ""Claudio Nicoletti "", … "" University of Pittsburgh United States ""]","[""OPEN ACCESS EDITED BY The glymphatic system: a new perspective on brain diseases"", ""OPEN ACCESS EDITED BY The glymphatic system: a new perspective on brain diseases""]","[""It is traditionally believed that the lymphatic system doesn't exist in the central nervous system (Nedergaard and Goldman, 2016). Thus, cell debris, potential neurotoxic proteins, and other metabolites with large molecular weight are considered to be removed in a different clearance pathway than brain vasculature. In 2012, the Nedergaard (Iliff et al., 2012) group found that cerebrospinal fluid (CSF) can enter brain parenchyma and exchange with brain interstitial fluid (ISF) in the presence of aquaporin-4 (AQP4) on astrocytes. Likewise, when the mixed fluid was drained from the brain, Amyloid-β (Aβ) was transported along with the outflow. Since the function of this ""drainage"" pathway is similar to that of the lymphatic system and supported by astrocytes, it was named the glymphatic system sooner after glia (Jessen et al., 2015)."", ""Since last decade, many researchers in the field of neurology, neurodegenerative diseases and physiology have aimed to study and develop the glymphatic system, providing a brandnew perspective for us to understand brain diseases: the overall fluid flow of the brain rather than a specific lesion or structure. Herein, we summarized the components of the glymphatic system, the fluid circulation mode within this system, how pathogenic solutes are transported in certain diseases, affecting factors of its function, and by what means we can study the glymphatic system. All above may provide direction and reference for brain diseases and medical researchers."", … ""This work was supported by the Funds of the scientific and technological innovation project of the Chinese Academy of Traditional Chinese Medicine CI2021A04614.""]"


Indexing
========
To avoid memory overflow we index everything by slices

In [8]:
moskalev.index_by_slices(10, db, 0)

2023-08-06 14:33:01.672 | INFO     | indexpaper.paperset:index_fun:168 - indexed 0 + 10 * 792 documents with last document having 10.3389/fphar.2016.00505
2023-08-06 14:35:12.323 | INFO     | indexpaper.paperset:index_fun:168 - indexed 0 + 10 * 508 documents with last document having 10.3389/fgene.2015.00353
2023-08-06 14:38:44.692 | INFO     | indexpaper.paperset:index_fun:168 - indexed 0 + 10 * 858 documents with last document having 10.18632/oncotarget.24461
2023-08-06 14:41:27.001 | INFO     | indexpaper.paperset:index_fun:168 - indexed 0 + 10 * 579 documents with last document having 10.3389/fgene.2019.00303
2023-08-06 14:43:45.845 | INFO     | indexpaper.paperset:index_fun:168 - indexed 0 + 10 * 561 documents with last document having 10.3389/fgene.2018.00095
2023-08-06 14:47:41.329 | INFO     | indexpaper.paperset:index_fun:168 - indexed 0 + 10 * 803 documents with last document having 10.3389/fmed.2020.00371
2023-08-06 14:50:21.931 | INFO     | indexpaper.paperset:index_fun:168

In [9]:
# Testing index


In [17]:
db.similarity_search("IKK-β activation ")[0].page_content

'proteins: p65 (RelA), RelB, c-Rel, p105/p50 (NF-κB1), and p100/52 (NF-κB2), which associate with each other to form distinct active NF-κB dimers. In the cytosol, NF-κB dimers in inactive form anchored by IκB are found. There are two ways to activate the NF-kB-induced gene transcription. These are triggered by cytokines TNFα and IL-1 (canonical signaling) or antigen receptors CD40 and BAFF (non-canonical/alternative signaling). There are the activation of IKK complex (IKKα, IKKβ, and IKKγ) and the phosphorylation of IκB proteins, which are in interaction with'